# Neural Network Model

## Import Requird Libraries

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim

## Build a model

In [39]:
class PredictRisk(nn.Module):
  def __init__(self, input_dim):
    super(PredictRisk, self).__init__()
    self.net = nn.Sequential(
        nn.Linear(input_dim,16),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(16,8),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(8,4),
        nn.ReLU(),
        nn.Dropout(p=0.4),

        nn.Linear(4,1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.net(x)

## Preparing Dataset

In [6]:
# importing required library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
# Read csv and create df
df = pd.read_csv("/content/drive/MyDrive/Data/Final Dataset.csv")
df.set_index('Patient ID', inplace=True)
df.head()

,Heart Rate,Body Temperature,Oxygen Saturation,Age,Gender,Weight,Height (m),HRV,BMI,Risk Category
Patient ID,,,,,,,,,,
1,60,36.861707,95.702046,37,0,91.541618,1.679351,0.121033,32.459031,0
2,63,36.511633,96.689413,77,1,50.704921,1.992546,0.117062,12.771246,0
3,63,37.052049,98.508265,68,0,90.316760,1.770228,0.053200,28.821069,1
4,99,36.654748,95.011801,41,0,96.006188,1.833629,0.064475,28.554611,0
5,69,36.975098,98.623792,25,0,56.020006,1.866419,0.118484,16.081438,0


In [13]:
# importing values
X = df.drop("Risk Category", axis=1).values
y = df["Risk Category"].values

# train_test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((160016, 9), (40004, 9), (160016,), (40004,))

In [18]:
# Scale the features
Scaler = StandardScaler()
Scaler.fit(X_train)
X_train_scaled = Scaler.transform(X_train)
X_test_scaled = Scaler.transform(X_test)

# convert to the tensor
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)   # add new dimension

X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)    # add new dimension

In [24]:
X_train_tensor.shape[1], y_train_tensor.shape

(9, torch.Size([160016, 1]))

## Instanting the model

In [54]:
my_model = PredictRisk(input_dim=X_train_tensor.shape[1])
loss_fn = nn.BCELoss()
optimizer = optim.Adam(my_model.parameters(),lr=0.001)

## Training loop

In [55]:
epochs = 400
for epoch in range(epochs):
  my_model.train()
  y_pred = my_model(X_train_tensor)
  loss = loss_fn(y_pred, y_train_tensor)

  # Backpropagation
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if (epoch+1) % 20 == 0:
    print(f"Epoch: {epoch+1}, loss: {loss.item():.4f}")

Epoch: 20, loss: 0.7105
Epoch: 40, loss: 0.6989
Epoch: 60, loss: 0.6853
Epoch: 80, loss: 0.6670
Epoch: 100, loss: 0.6450
Epoch: 120, loss: 0.6165
Epoch: 140, loss: 0.5853
Epoch: 160, loss: 0.5544
Epoch: 180, loss: 0.5284
Epoch: 200, loss: 0.5061
Epoch: 220, loss: 0.4873
Epoch: 240, loss: 0.4698
Epoch: 260, loss: 0.4536
Epoch: 280, loss: 0.4379
Epoch: 300, loss: 0.4213
Epoch: 320, loss: 0.4036
Epoch: 340, loss: 0.3840
Epoch: 360, loss: 0.3649
Epoch: 380, loss: 0.3433
Epoch: 400, loss: 0.3247


## Testing loop

In [56]:
my_model.eval()
with torch.no_grad():
  preds = my_model(X_test_tensor)
  preds_class = (preds > 0.5).float()
  accuracy = (preds_class.eq(y_test_tensor).sum().item())/y_test_tensor.shape[0]

print("Test Accuracy: ", accuracy)

Test Accuracy:  0.978077192280772


In [57]:
# confusion matrix
y_true = y_test_tensor.numpy()
y_pred = preds_class.numpy()

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true, y_pred)
print(cm)

[[20893   130]
 [  747 18234]]


## Save the model

In [58]:
torch.save(my_model.state_dict(),'NeuralModel.pth')
torch.save(optimizer.state_dict(), 'Neuraloptimizer.pth')